In [1]:
import numpy as np
import cv2
import os

import itertools

from evaluation_scripts.metrics import calculate_angle, read_from_json
from src.vanishingpoint import run

from test_generation.test_generation_custom import generate_ds

In [2]:
path2res = '/Users/kgb/Google Drive/Code/VanishingPoint/res/ans'
path2res_folder = '/Users/kgb/Google Drive/Code/VanishingPoint/res/'
path2train_gt = '/Users/kgb/Google Drive/Code/VanishingPoint/train/markup.json'
path2train_data = '/Users/kgb/Google Drive/Code/VanishingPoint/train/'
path2test_data = '/Users/kgb/Google Drive/Code/VanishingPoint/test/'
path2test_gt = '/Users/kgb/Google Drive/Code/VanishingPoint/test/markup.json'

In [3]:
def get_shapes(path):
    names = os.listdir(path)
    return {name: cv2.imread(os.path.join(path, name)).shape[0:2] for name in names if name != 'markup.json'}

def generate_test_data(num = 50, seed = 9876):
    return generate_ds(path2test_data, path2train_data, num, seed)

In [4]:
selected_fnames = generate_test_data(5)
shapes_dict = get_shapes(path2test_data)
y_true_dict = read_from_json(path2test_gt)

def evaluation(sigma, threshold_dividend, min_dividend, max_dividend):
    t_res_dict = {}

    for f in selected_fnames:
        try:
            t_res_dict[f] = run(path2test_data + f'{f}',
                              sigma,
                              threshold_dividend,
                              min_dividend,
                              max_dividend)
        except Exception as e:
            print(f)
            print(e)

    return t_res_dict

In [5]:
def calc_metrics(parameters):
    sigma = parameters[0]
    threshold_dividend = parameters[1]
    min_dividend = parameters[2]
    max_dividend = parameters[3]

    y_predict_dict = evaluation(sigma, threshold_dividend, min_dividend, max_dividend)

    angles = []
    for name in y_true_dict:
        angle = calculate_angle(y_predict_dict[name], y_true_dict[name], shapes_dict[name])
        angles.append(angle)

    angles = np.array(angles)

    return np.mean(angles), np.median(angles)

In [6]:
x0 = [0.2, 23, 150.0, 150.0]
calc_metrics(x0)

(3.537878101918582, 3.1170727239945153)

In [14]:
sigmas = [0.2]
thresholds = [20, 24, 25, 30, 40, 50, 60, 75, 100, 120, 150, 200, 300, 600]
min_dividends = [1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 25, 30, 50, 60, 75, 100, 150, 300]
max_dividends = [1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 25, 30, 50, 60, 75, 100, 150, 300]

products = itertools.product(sigmas, thresholds, min_dividends)

len(list(products))

252

In [8]:
res_dict = {}

for sigma, threshold, min_dividend in products:
    pos_max_dividends = [value for value in max_dividends if value > min_dividend]
    for max_dividend in pos_max_dividends:
        t_res = calc_metrics([sigma, threshold, min_dividend, max_dividend])
        res_dict[(sigma, threshold, min_dividend, max_dividend)] = t_res
        print(t_res)

In [9]:
res_dict_sorted = dict(sorted(res_dict.items(), key=lambda item: item[1][0] + item[1][1]))
res_dict_sorted